# Classificatie - Logistic Regression

## Wat is classificatie?

Bij regressie wordt er getracht een manier te zoeken om op basis van een aantal inputs/features een continue waarde te voorspellen.
Bij classificatie omvat machine learning technieken waar een model getrained wordt dat op basis van de input bepaalt in welke categorieen je input thuishoort.
Een model dat een categorie toekent aan een bepaalde set input-data kan ook een **classifier** genoemd worden.
Het classificatieprobleem hoort nog steeds thuis onder **supervised learning** omdat er nog steeds gebruikt gemaakt wordt van een output/label/target om het model te trainen.

Enkele voorbeelden van classificatieproblemen zijn
* Spam detectie
* Medische diagnoses
* Kwaliteitscontrole
* Geschreven tekst inlezen
* ...


## Types van classifiers

Op basis van hoe de data gelabeled wordt kunnen we drie types van classifiers onderscheiden:
* Binaire classifier: Er zijn twee verschillende klasses (goedgekeurd of niet, kwaadaardig vs goedaardig, ...). Lijkt simplistisch maar komt heel veel voor.
* Multiclass classifier: Er zijn een willekeurig aantal klassen en er kan er maar 1 toegekend worden. Een voorbeeld hiervan is gezichtsherkenning (elk gezicht kan maar bij 1 persoon horen).
* Multilabel classifier: Er zijn een willekeurig aantal klassen en 1 input kan tot meerdere klassen behoren, bijvoorbeeld op een foto kan je meerdere zaken herkennen zoals een boom, wolken, gras, struiken, dieren, ...

## Kan hier ook lineaire regressie gebruikt worden?

Een eerste mogelijkheid om classificatie uit te voeren is om elke klasse een apart getal te geven en dan lineaire regressie uit te voeren.
Hierdoor bekomen we een getal en dan kunnen we kijken naar welk cijfer/klasse er het dichts bij licht.
De problemen met deze aanpak kunnen het makkelijkst worden toegelicht doorm iddel van een voorbeeld.
Doorheen deze notebook gaat er grotendeels gewerkt worden met [deze dataset](https://www.kaggle.com/elakiricoder/gender-classification-dataset) die 7 zaken gemeten heeft in het gezicht van zowel mannen als vrouwen.
Deze dataset gaan we gebruiken om op basis van deze metingen te gaan bepalen of de bijhorende persoon een vrouw of man is.

In [1]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import opendatasets as od

# sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

%matplotlib inline

In [ ]:
od.download("https://www.kaggle.com/joshmcadams/oranges-vs-grapefruit")

In [10]:
df = pd.read_csv(".\\oranges-vs-grapefruit\\citrus.csv")

In [ ]:
df["output"] = df.name == "orange"
df

Waneer we nu een scatterplot tekenen van een kleine datasample en lineaire regressie zouden laten uitvoeren bekomen we onderstaand resultaat.
Voor classificatie zouden we beslissen dat wanneer het minder is dan de helft, dat het dan een orange is en anders pompelmoes.

In [ ]:
def split(group):
    s = group.sort_values("weight")
    if group.name == "orange":
        return s.head()
    return s.tail()
df_trimmed = df.groupby(["name"]).apply(split)

model = LinearRegression()
model.fit(pd.DataFrame(df_trimmed.diameter), df_trimmed.output)
x_test = [4, 16]
y_test = model.predict(pd.DataFrame(x_test))

sns.scatterplot(x = 'diameter', y ='output', data=df_trimmed, hue='output')
plt.plot(x_test, y_test)

Met deze plot zou je kunnen denken dat lineaire regressie kan gebruikt worden.
Daarom hebben we eens een andere groep ook geplot en daar zie je dat er toch een aantal problemen zijn met lineaire regressie te gebruiken voor classificatie.
* Outliers hebben een grote impact op de trendlijn en dus op de manier dat de classificatie gebeurd
* Als de ene klas 0 is en de andere 1, dan kunnen we waarden voorspellen die kleiner zijn dan 0 en groter dan 1. Dit komt niet overeen met kansen dus is het lastiger om te interpreteren. 
* Als de klassen niet gebalanceerd zijn kan het zijn dat er een grotere bias is voor de ene klassen. Dit wordt ook hieronder. aangetoond.

De combinatie van deze problemen zorgt ervoor dat lineaire regressie zelden een goede oplossing is voor classificatieproblemen.
Gelukkig bevat het domein van classificatie een hele hoop machine learning technieken die beter bestand zijn tegen deze problemen.
In de loop van de cursus gaan we er een aantal van bestuderen, namelijk:
* Logistic Regression
* SVM
* Naive Bayes
* Decision Trees
* Random Forest

In [ ]:
def split_outlier(group):
    s = group.sort_values("weight")
    if group.name == "orange":
        return s.head()
    return pd.concat([s.tail(1), s[-1000:-995]])
df_trimmed = df.groupby(["name"]).apply(split_outlier)

model = LinearRegression()
model.fit(pd.DataFrame(df_trimmed.diameter), df_trimmed.output)
x_test = [4, 16]
y_test = model.predict(pd.DataFrame(x_test))

sns.scatterplot(x = 'diameter', y ='output', data=df_trimmed, hue='output')
plt.plot(x_test, y_test)

In [ ]:
def split_inbalanced(group):
    s = group.sort_values("weight")
    if group.name == "orange":
        return s.head()
    return s.tail(2000)
df_trimmed = df.groupby(["name"]).apply(split_inbalanced)

model = LinearRegression()
model.fit(pd.DataFrame(df_trimmed.diameter), df_trimmed.output)
x_test = [4, 16]
y_test = model.predict(pd.DataFrame(x_test))

sns.scatterplot(x = 'diameter', y ='output', data=df_trimmed, hue='output')
plt.plot(x_test, y_test)

In [ ]:
def split_inbalanced(group):
    s = group.sort_values("weight")
    if group.name == "orange":
        return s.head()
    return s.tail(2000)
df_trimmed = df.groupby(["name"]).apply(split_inbalanced)

model = LogisticRegression(penalty=None)
model.fit(pd.DataFrame(df_trimmed.diameter), df_trimmed.output)
x_test = np.linspace(4,16,1000)
y_test = model.predict(pd.DataFrame(x_test))

sns.scatterplot(x = 'diameter', y ='output', data=df_trimmed, hue='output')
plt.plot(x_test, y_test)

## Evaluatie van classificatie

In tegenstelling tot lineaire regressie kunnen we niet de afstand tot de gewenste output gebruiken om verschillende classifiers te gaan evalueren.
Een mogelijkheid die kan werken is om te kijken naar het percentage van inputs die correct geklassificieerd zijn.
Dit wordt ook de accuraatheid of accuracy genoemd.
Enkel deze metriek gebruiken heeft echter problemen bij het classificieren van zeldzame events of niet-gebalanceerde data.
Een classifier zou bijvoorbeeld voor iets dat in maar 1 van de 1000 gevallen voorkomt kunnen voorspellen dat het nooit voorkomt maar toch een accuraatheid bekomen van 99,9%.

Om dit op te vangen kan de zogenoemde [confusionmatrix](https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826) bepaald worden.
Hierin wordt bijgehouden voor elke klasse hoeveel voorspellingen er correct waren en hoeveel niet.
Op basis van deze matrix kunnen de volgende metrieken berekend worden:
* Accuraatheid: algemene correctheid $\frac{TP+TN}{TP+TN+FP+FN}$
* Precisie: Accuraatheid binnen positieve voorspellingen $\frac{TP}{TP+FP}$
* Specificiteit: Accuraatheid binnen negatieve samples $\frac{TN}{TN+FP}$
* Recall: Accuraatheid binnen positieve samples $\frac{TP}{TP+FN}$
* F1-score: Combinatie Precisie en Recall: $2\frac{Precisie*Recall}{Precisie+Recall} = \frac{2TP}{2TP+FP+FN}$

De [.score](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) functie van het LogisticRegression model berekent de accuraatheid van het model maar de andere metrieken kunnen een veel nauwkeuriger beeld geven over de correcte werken van het model.

![matrix](images/confusionMatrix.png)

## Multi-class classification

In het bovenstaande voorbeeld zijn de training examples onderverdeeld in twee klassen.
In de praktijk komt het ook vaak voor dat er meerdere klasses mogelijk zijn, zoals meerdere types fruit, meerdere soorten hondenrassen, ...
Om dit aan te pakken zijn er twee veelgebruikte technieken: One vs All en One vs One.
Meer informatie over hoe de LogisticRegression van sklearn kan gebruikt worden voor multi-class problemen op te lossen vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Dan gaat het specifiek om het multi-class probleem.

### One vs All

Bij deze techniek maak 1 classifier per klasse die je hebt.
Deze classifier bepaalt of een input van die bepaalde klasse is of niet.
Met deze techniek worden er dus N classifiers getrained.
Het voordeel hiervan is dat er minder classifiers nodig zijn dan in de One-Vs-One-techniek waardoor het sneller te trainen is.
Het nadeel echter is dat de techniek gevoeliger is aan niet-gebalanceerde data.
Deze techniek wordt ook vaak One-Vs-The-Rest genoemd.

Een opmerking bij techniek is dat het mogelijk is dat twee of meerdere classifiers aangeven dat een input tot die klasse hoort.
In het geval dat er een dergelijk conflict opduikt wordt de klasse die de hoogste kans aangeeft gekozen. 

![onevsall](images/oneVsAll.png)

### One vs One

Bij deze techniek maken we een classifier aan voor elke combinatie van 2 klassen aan. 
Hierdoor zijn er $\frac{N(N-1)}{2}$ classifiers nodig wat snel kan oplopen in vergelijking met de voorgaande techniek.
Op basis van onderstaand voorbeeld werkt deze techniek als volgt:
* Eerst wordt er gekeken of de input van de rode of de blauwe klasse is. 
* Indien het niet van de rode is, wordt er nog gekeken of het blauw of groen is.
* Indien het niet de blauwe klasse is, wordt er nog gekeken of het rood of groen is.

Hoewel deze methode een grotere trainingstijd nodig heeft door het toegenomen aantal classifiers zijn er wel een aantal voorbeelden:
* Minder gevoelig voor niet gebalanceerde data
* Geen wazige gebieden waar meerdere classifiers kunnen voorspellen dat het van die klasse is.

![onevsall](images/oneVsOne.png)

### Multinomial

Sommige van de solvers in logistic regression maken het ook mogelijk om rechtstreeks te werken met meerdere klassen. In deze gevallen kan het "multi-class" argument op "multinomial" gezet worden. Dit is het standaard gedrag tenzij de solver op "liblinear" staat.

In [ ]:
OneVsOneClassifier()
OneVsRestClassifier()

### Multiclass evaluatie

De confusion matrix in het geval van meerdere klassen kan uitgebreid worden naar onderstaand voorbeeld voor drie klassen. 

![matrix](images/confusionMatrixMulti.png)

Hier is het niet rechtstreeks zichtbaar wat de True Positive, True Negative, ... waarden zijn.
We kunnen deze echter eenvoudig berekenen per klasse. 
Dan kunnen we de besproken metrieken ook per klasse berekenen zodat we onderstaande resultaten bekomen.

![matrix](images/metriekenMulti.png)

Met behulp van deze matrix kunnen verschillende modellen vergeleken worden.
Echter kan er nog iets verder gegaan worden in het combineren van waarden door een globale F1-score te bepalen.
Hier zijn meerdere opties voor, namelijk:
* Micro F1: Globale F1 op basis van total TP, FN en FP. Merk op dat door geen onderscheid te maken tussen klassen dat $Micro F1 = Precisie = Recall = Accuraatheid$
* Macro F1: (Niet gewogen) Gemiddelde van alle F1-scores per klasse.
* Weighted F1: Gewogen gemiddelde van de F1-scores per klasse (gewichten = aantal samples van die klasse)

## Voorbeeld

Gebruik nu [deze dataset](https://www.kaggle.com/c/titanic/overview) om onderstaande vragen op te lossen:
* Maak een model om op basis van de kolommen te voorspellen of een bepaalde passagier de tocht op de titanic zou overleven
* Bereken voor dit model de accuraatheid, recall, precisie en F1-score. 
* Probeer hogere-orde features te gebruiken om deze waarden te verhogen
* Maak een scatter plot voor de leeftijd vs hoeveel er betaald was voor het ticket. Geef ook aan welke datapunten de ramp overleefd hebben en welke niet.
* Maak nu een one-vs-all en one-vs-one classifier aan om de klasse van een persoon te voorspellen op basis van de andere gegevens. Gebruik hiervoor niet de testset omdat hier geen data voor beschikbaar is in de testset maar splits je trainingsset op in twee delen.
* Schrijf nu een functie om de confusion matrix en de metrieken per klasse en de gecombineerde F1-scores te bepalen. Zoek naar het beste model.
* Welke hyperparameters heb je gebruikt tijdens het zoeken naar het model? Hoeveel parameters heeft je uiteindelijke model?

In [ ]:
od.download("https://www.kaggle.com/c/titanic/overview")